## Install Flask to enable network communication (micro-services)

### Start up the service

In [ ]:
!python fraud_detection_service.py


Try out the links 

* [determination](http://localhost:8786/infer?check_transaction=check_transaction.csv)

Paste the following link or click on it

http://localhost:8786/infer?check_transaction=check_transaction.csv

### You must kill the kernel to try again for the port stays locked to the current kernel